<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/lease_rests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install censusgeocode
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys
from rets import Session

In [6]:
def lease_findifNull(lo_left,lo_right,la_left,la_right,research,zip):
  result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (PostalCode = {})'.format(lo_left, lo_right, la_left, la_right, zip))
  lis = []
  count = 0
  di ={}
  df = pd.DataFrame()
  for item in result:
    if(item['PropertyType'] == 'Residential Lease' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) #?
            count += 1
            di = item
  for key in di:
    lis.append(key)
  if(count >= 5): #not posible larger than 5 at the first time
    df.columns = lis
  return df


In [ ]:
def lease_findHouse():
  return

In [ ]:
def main():
  

In [ ]:
if __name__ == "__main__":
    print(main())